In [1]:
import gym
import numpy as np

In [2]:
env = gym.make('FrozenLake8x8-v0')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
def epsilon_greedy_policy(Q,state,epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(Q.shape[1])
    else:
        return Q[state,:].argmax()

In [4]:
def n_step_sarsa(env,n,n_episodes,alpha,gamma,epsilon):
    
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    
    Q = np.zeros((n_states,n_actions))+0.1
    
    for episode in range(n_episodes):
        S = {}
        A = {}
        R = {}
        
        t = 0
        T = np.inf      
        
        state = env.reset()
        
        while True:
            if t < T:
                action = epsilon_greedy_policy(Q,state,epsilon)
                S[t] = state
                A[t] = action
                state, reward, done, info = env.step(action)
                R[t] = reward

                if done:
                    T = t+1
            
            tau = t-n
            if tau >= 0:
                G = np.sum([gamma**(i-tau-1)* R[i] for i in range(tau+1,min(tau+n,T))])
                if tau+n<T:
                    G += gamma**n * Q[S[tau+n],A[tau+n]]
                Q[S[tau],A[tau]] += alpha * (G - Q[S[tau],A[tau]])
            
            t += 1
            if tau == T-1:
                break
    Q[Q==0.1] = 0
    return Q

In [5]:
Q = n_step_sarsa(env,3,20000,0.5,0.9,0.1)

In [6]:
Q

array([[1.84714821e-04, 1.70943889e-04, 1.79589478e-04, 8.68962098e-05],
       [2.73896384e-04, 2.72804406e-04, 2.42962004e-04, 4.66783971e-04],
       [2.90354746e-04, 6.15865570e-04, 3.04267962e-04, 2.70970896e-04],
       [6.72290199e-04, 9.09144002e-04, 1.15297885e-03, 1.48861346e-03],
       [1.33065481e-03, 1.17862427e-03, 1.41717087e-03, 3.15070340e-04],
       [1.64718529e-03, 1.55714338e-03, 1.74575960e-03, 1.62099422e-03],
       [2.08421943e-03, 2.09661018e-03, 2.09484655e-03, 2.41871796e-03],
       [1.86204973e-02, 2.32465622e-03, 1.92921428e-03, 2.08671767e-03],
       [1.61695510e-04, 1.69513987e-04, 1.90920920e-04, 1.80055650e-04],
       [1.06425443e-04, 1.94195034e-04, 1.95139695e-04, 4.08382129e-04],
       [3.23107293e-04, 3.08330218e-04, 2.59394462e-04, 4.04463741e-04],
       [4.80059848e-04, 3.06517900e-05, 1.12574476e-04, 4.51671561e-04],
       [1.65872436e-03, 1.78258475e-04, 1.44798169e-03, 1.64658692e-03],
       [8.03934668e-04, 1.87744654e-03, 1.58678295e

### TEST

In [7]:
def test(n_episodes,Q):
    wins = 0
    for i_episode in range(n_episodes):
        observation = env.reset()
        for t in range(100):
            action = Q[observation,:].argmax()
            observation, reward, done, info = env.step(action)
            if done:
                if reward != 0:
                    wins += 1
                break
    return wins

In [8]:
test(20,Q)

2